In [13]:
from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig, DefaultModelMonitor, CronExpressionGenerator
import sagemaker
from sagemaker import image_uris, get_execution_role, session

sess = session.Session()
role = get_execution_role()

image_uri = image_uris.retrieve('xgboost', sess.boto_region_name, version='latest')
# Define the model
model = Model(
    image_uri=image_uri,  # Replace with your image URI
    model_data='s3://electronics-dataset/sagemaker/benchmark-model/output/xgboost-2024-06-30-22-29-23-246/output/model.tar.gz',  # Replace with your model data path
    role=role,
    sagemaker_session=sess
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count = 1,
    instance_type='ml.m5.xlarge',
    endpoint_name='monitoring-control'  # Replace with your endpoint name
)

# Define data capture configuration
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri='s3://electronics-dataset/sagemaker/',  # Replace with your S3 path for data capture
    capture_options=['REQUEST', 'RESPONSE']
)

# Update the endpoint with data capture configuration
predictor.update_endpoint(
    data_capture_config=data_capture_config
)

# Define the monitoring schedule
schedule_config = ScheduleConfig(
    schedule_expression='cron(0 1 * * ? *)'  # Adjust the schedule as needed
)

# Create a ModelMonitor instance
model_monitor = ModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://your-bucket/monitoring-output/'  # Replace with your S3 path for monitoring output
)

# Set up the monitoring job
monitoring_job = model_monitor.create_monitoring_schedule(
    endpoint_name='monitoring-control',  # Replace with your endpoint name
    monitoring_schedule_config=schedule_config
)

print("Monitoring schedule created:", monitoring_job)


ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:676076160400:endpoint-config/monitoring-control".